In [1]:
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
import pandas as pd
from datasets import load_dataset, DatasetDict, load_dataset
from transformers.integrations import TensorBoardCallback
from torch.utils.tensorboard import SummaryWriter

/Users/devakrishna/opt/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
bert = AutoModel.from_pretrained('ai4bharat/indic-bert')

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.LayerNorm.bias', 'predictions.dense.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.bias', 'predictions.decoder.bias', 'predictions.bias', 'predictions.decoder.weight', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
model = AutoModelForSequenceClassification.from_pretrained('ai4bharat/indic-bert', num_labels=2)

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.dense.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.bias', 'predictions.decoder.bias', 'predictions.bias', 'predictions.decoder.weight', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [39]:
dataset = load_dataset('csv', data_files={'train': ['./dataset/train.csv',],
                                              'test': './dataset/validation.csv'})


Using custom data configuration default-3d8409e80e43e3ab


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /Users/devakrishna/.cache/huggingface/datasets/csv/default-3d8409e80e43e3ab/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [41]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=100)

In [42]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [43]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [50]:
from transformers import TrainingArguments

training_args = TrainingArguments("logging_steps")
training_args.logging_steps = 5
training_args.num_train_epochs = 50

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [45]:
class MyCallback(TensorBoardCallback):
    pass

In [46]:
model_name = "hugginface_fine_tune"
writer = SummaryWriter(f'runs/{model_name}')

In [48]:
trainer = Trainer(
    model=model,\
    args=training_args,\
    train_dataset=full_train_dataset,\
    eval_dataset=full_eval_dataset,\
    callbacks=[MyCallback(writer)]
)

In [49]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 4704
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5880


Step,Training Loss
5,0.666300


KeyboardInterrupt: 